<div style='text-align: center;'>
    <img title="Banner" alt="Logo ITESO" src="https://oci02.img.iteso.mx/Identidades-De-Instancia/ITESO/Logos%20ITESO/Logo-ITESO-Principal-FondoAzul.png" width=600 align='center'>
</div>

# Proyecto 2 - Minería de Grafos
## Dr. Victor Hugo Ortega Guzmán

* Sofía Maldonado García
* Viviana Toledo De la Fuente
* César Adrián Santos Santacruz




In [1]:
# Actualizar 
import importlib
import queries 
importlib.reload(queries)
from queries import DB_Queries

# Establecer conexión
db = DB_Queries()

ConfigurationError: URI scheme b'' is not supported. Supported URI schemes are ['bolt', 'bolt+ssc', 'bolt+s', 'neo4j', 'neo4j+ssc', 'neo4j+s']. Examples: bolt://host[:port] or neo4j://host[:port][?routing_context]